# Absolutemomentum Strategy

In [2]:
import pandas as pd
import numpy as np
import datetime
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
plt.style.use(['fivethirtyeight'])

# pd.set_option('display.max_seq_items', None)
# pd.set_option('display.max_rows', None)


In [3]:
dataframe = fdr.DataReader('005930')     # Index is already 'Date'
price_df = dataframe.loc[:, ['Close']].copy()    # Extract 'Close' column, 'fdr' offer Adjusted Close as 'Close'

price_df['STD_YM'] = price_df.index.map(lambda x : pd.Timestamp(x + pd.offsets.MonthEnd(n=0)).strftime('%Y-%m'))
price_df.reset_index(inplace=True)
price_df

,Date,Close,STD_YM
0,1997-08-30,1260,1997-08
1,1997-09-01,1251,1997-09
2,1997-09-02,1269,1997-09
3,1997-09-03,1237,1997-09
4,1997-09-04,1221,1997-09
...,...,...,...
5995,2021-09-10,75300,2021-09
5996,2021-09-13,76300,2021-09
5997,2021-09-14,76600,2021-09
5998,2021-09-15,77000,2021-09


In [4]:
month_list = price_df['STD_YM'].unique()
month_last_df = pd.DataFrame()
for m in month_list:
    month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == m].index[-1], : ])

month_last_df.set_index(['Date'], inplace=True)
print(month_last_df.head(15))

             Close   STD_YM
Date                       
1997-08-30  1260.0  1997-08
1997-09-30  1188.0  1997-09
1997-10-31   677.0  1997-10
1997-11-29   756.0  1997-11
1997-12-27   686.0  1997-12
1998-01-31  1616.0  1998-01
1998-02-28  1852.0  1998-02
1998-03-31  1325.0  1998-03
1998-04-30  1378.0  1998-04
1998-05-30   936.0  1998-05
1998-06-30   791.0  1998-06
1998-07-31   847.0  1998-07
1998-08-31   860.0  1998-08
1998-09-30   728.0  1998-09
1998-10-31  1149.0  1998-10


In [5]:
month_last_df['BF_1M_Close'] = month_last_df.shift(1)['Close']
month_last_df['BF_12M_Close'] = month_last_df.shift(12)['Close']
month_last_df.fillna(0, inplace=True)
print(month_last_df.head(15))

             Close   STD_YM  BF_1M_Close  BF_12M_Close
Date                                                  
1997-08-30  1260.0  1997-08          0.0           0.0
1997-09-30  1188.0  1997-09       1260.0           0.0
1997-10-31   677.0  1997-10       1188.0           0.0
1997-11-29   756.0  1997-11        677.0           0.0
1997-12-27   686.0  1997-12        756.0           0.0
1998-01-31  1616.0  1998-01        686.0           0.0
1998-02-28  1852.0  1998-02       1616.0           0.0
1998-03-31  1325.0  1998-03       1852.0           0.0
1998-04-30  1378.0  1998-04       1325.0           0.0
1998-05-30   936.0  1998-05       1378.0           0.0
1998-06-30   791.0  1998-06        936.0           0.0
1998-07-31   847.0  1998-07        791.0           0.0
1998-08-31   860.0  1998-08        847.0        1260.0
1998-09-30   728.0  1998-09        860.0        1188.0
1998-10-31  1149.0  1998-10        728.0         677.0


In [6]:
book = price_df.copy()
book.set_index(['Date'], inplace=True)
book['trade'] = ''
print(book.head())

            Close   STD_YM trade
Date                            
1997-08-30   1260  1997-08      
1997-09-01   1251  1997-09      
1997-09-02   1269  1997-09      
1997-09-03   1237  1997-09      
1997-09-04   1221  1997-09      


In [8]:
ticker = 'SEC' # Samsung Electronics
for x in month_last_df.index[12:]:
    signal = ''
    momentum_index = month_last_df.loc[x, 'BF_1M_Close'] / month_last_df.loc[x, 'BF_12M_Close'] - 1
    if (momentum_index > 0.0) and (momentum_index != np.inf) and (momentum_index != -np.inf):
        flag = True
    else:
        flag = False
    if flag:
        signal = 'buy ' + ticker
    print('날짜 : ', x, ' 모멘텀 인덱스 : ', momentum_index, '   \tflag : ', flag, 'signal : ', signal)
    book.loc[x:, 'trade'] = signal

날짜 :  1998-08-31 00:00:00  모멘텀 인덱스 :  -0.3277777777777777    	flag :  False signal :  
날짜 :  1998-09-30 00:00:00  모멘텀 인덱스 :  -0.27609427609427606    	flag :  False signal :  
날짜 :  1998-10-31 00:00:00  모멘텀 인덱스 :  0.07533234859675031    	flag :  True signal :  buy SEC
날짜 :  1998-11-30 00:00:00  모멘텀 인덱스 :  0.5198412698412698    	flag :  True signal :  buy SEC
날짜 :  1998-12-28 00:00:00  모멘텀 인덱스 :  0.8600583090379008    	flag :  True signal :  buy SEC
날짜 :  1999-01-29 00:00:00  모멘텀 인덱스 :  -0.043316831683168355    	flag :  False signal :  
날짜 :  1999-02-26 00:00:00  모멘텀 인덱스 :  -0.05021598272138228    	flag :  False signal :  
날짜 :  1999-03-31 00:00:00  모멘텀 인덱스 :  0.2732075471698112    	flag :  True signal :  buy SEC
날짜 :  1999-04-30 00:00:00  모멘텀 인덱스 :  0.34905660377358494    	flag :  True signal :  buy SEC
날짜 :  1999-05-31 00:00:00  모멘텀 인덱스 :  0.9102564102564104    	flag :  True signal :  buy SEC
날짜 :  1999-06-30 00:00:00  모멘텀 인덱스 :  1.0960809102402025    	flag :  True signal :  buy SEC
날짜

날짜 :  2013-01-31 00:00:00  모멘텀 인덱스 :  0.37488708220415545    	flag :  True signal :  buy SEC
날짜 :  2013-02-28 00:00:00  모멘텀 인덱스 :  0.20066334991708135    	flag :  True signal :  buy SEC
날짜 :  2013-03-29 00:00:00  모멘텀 인덱스 :  0.2109803921568627    	flag :  True signal :  buy SEC
날짜 :  2013-04-30 00:00:00  모멘텀 인덱스 :  0.09856115107913666    	flag :  True signal :  buy SEC
날짜 :  2013-05-31 00:00:00  모멘텀 인덱스 :  0.2551610239471511    	flag :  True signal :  buy SEC
날짜 :  2013-06-28 00:00:00  모멘텀 인덱스 :  0.28059950041631976    	flag :  True signal :  buy SEC
날짜 :  2013-07-31 00:00:00  모멘텀 인덱스 :  0.025210084033613356    	flag :  True signal :  buy SEC
날짜 :  2013-08-30 00:00:00  모멘텀 인덱스 :  0.03811841038118402    	flag :  True signal :  buy SEC
날짜 :  2013-09-30 00:00:00  모멘텀 인덱스 :  0.01634472511144125    	flag :  True signal :  buy SEC
날짜 :  2013-10-31 00:00:00  모멘텀 인덱스 :  0.04351145038167936    	flag :  True signal :  buy SEC
날짜 :  2013-11-29 00:00:00  모멘텀 인덱스 :  0.041963015647226154    	flag :  

In [9]:
def returns(book, ticker):
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        if book.loc[i, 'trade'] == 'buy ' + ticker and book.shift(1).loc[i, 'trade'] == '':
            buy = book.loc[i, 'Close']
            print('진입일 : ', i, 'long 진입가격 : ', buy)
        elif book.loc[i, 'trade'] == 'buy ' + ticker and book.shift(1).loc[i, 'trade'] == 'buy ' + ticker:
            current = book.loc[i, 'Close']
            rtn = (current - buy) / buy + 1
            book.loc[i, 'return'] = rtn
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy ' + ticker:
            sell = book.loc[i, 'Close']
            rtn = (sell - buy) / buy + 1
            book.loc[i, 'return'] = rtn
            print('청산일 : ', i, 'long 진입가격 : ', buy, ' | long 청산가격 : ', sell, ' | return:', round(rtn, 4))
        
        if book.loc[i, 'trade'] == '':
            buy = 0.0
            sell = 0.0
            current = 0.0
            
        acc_rtn = 1.0
    for i in book.index:
        if book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy ' + ticker:
            rtn = book.loc[i, 'return']
            acc_rtn = acc_rtn * rtn
            book.loc[i:, 'acc return'] = acc_rtn
                
    print('Accumulated return :', round(acc_rtn, 4))
    return (round(acc_rtn, 4))

In [10]:
returns(book, ticker)

진입일 :  1998-10-31 00:00:00 long 진입가격 :  1149
청산일 :  1999-01-29 00:00:00 long 진입가격 :  1149  | long 청산가격 :  1759  | return: 1.5309
진입일 :  1999-03-31 00:00:00 long 진입가격 :  1859
청산일 :  2000-11-30 00:00:00 long 진입가격 :  1859  | long 청산가격 :  3240  | return: 1.7429
진입일 :  2001-11-30 00:00:00 long 진입가격 :  4360
청산일 :  2003-02-28 00:00:00 long 진입가격 :  4360  | long 청산가격 :  5590  | return: 1.2821
진입일 :  2003-07-31 00:00:00 long 진입가격 :  8320
청산일 :  2004-08-31 00:00:00 long 진입가격 :  8320  | long 청산가격 :  9020  | return: 1.0841
진입일 :  2004-09-30 00:00:00 long 진입가격 :  9160
청산일 :  2004-10-29 00:00:00 long 진입가격 :  9160  | long 청산가격 :  8790  | return: 0.9596
진입일 :  2005-06-30 00:00:00 long 진입가격 :  9880
청산일 :  2006-12-28 00:00:00 long 진입가격 :  9880  | long 청산가격 :  12260  | return: 1.2409
진입일 :  2008-02-29 00:00:00 long 진입가격 :  11200
청산일 :  2008-03-31 00:00:00 long 진입가격 :  11200  | long 청산가격 :  12460  | return: 1.1125
진입일 :  2008-04-30 00:00:00 long 진입가격 :  14220
청산일 :  2008-08-29 00:00:00 long 진입가격 :  14220  

8.4447